In [1]:
'''Biblio'''

#Webdriver de Selenium qui permet de contrôler un navigateur
from selenium import webdriver

#Permet d'accéder aux différents élements de la page web
from selenium.webdriver.common.by import By

#Pour attendre qu'une condition soit remplie avant de poursuivre l'exécution du script
from selenium.webdriver.support.ui import WebDriverWait

#fournit des conditions d'attente prédéfinies pour être utilisées avec WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.support.ui import Select


import matplotlib.pylab as plt
import numpy as np

import re #used for Regular Expression 
import string
from collections import defaultdict

from collections import Counter
from datetime import datetime


from sklearn.feature_extraction.text import CountVectorizer

#Pour utiliser les fonctionnalités liées à la gestion du temps
import time

import re

from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

import pandas as pd

from datetime import datetime

from pprint import pprint 


from bs4 import BeautifulSoup


In [11]:
#=============================================================

'''sous - Fonction '''

def twitter_time(soup1):
    times = soup1.find_all('time')
    dates=[]
    for time in times:
        dates.append(time.get('datetime'))
    return dates

#

def twitter_user(soup1):
    users = soup1.find_all('div',attrs={'data-testid':'User-Name'})

    username=[]
    for div in users:
        divss = div.find('a')
        print(divss['href'])
        username.append(divss.get('href')[1:])
    return username


def twitter_tweets(soup1):
    tweets = soup1.find_all('div',attrs={'data-testid':"tweetText"})

    texts=[]
    for tweet in tweets:
        texts.append(tweet.text)
    return texts

def twitter_react(soup1):
    likes = soup1.find_all('div',attrs={'role':'group'})

    reacts=[]
    order = ['replies', 'reposts', 'likes', 'bookmarks', 'views']
    
    for like in likes:
        react_text=like['aria-label']

        # Initialize dictionary to store numbers
        react_stat = {word: 0 for word in order}

        # Find all numbers and words using regex
        matches = re.findall(r"(\d+) (\w+)", react_text)

        # Update numbers dictionary with extracted values
        for match in matches:
            number, word = match
            if word in react_stat:
                react_stat[word] = int(number)
        reacts.append([react_stat[word] for word in order])

        
    react_sorted = [[row[i] for row in reacts] for i in range(len(reacts[0]))]
        
    return react_sorted

def exctract_twitter_data(soup1):
    dictio = {}
    dictio['Username']=twitter_user(soup1)
    dictio['Date']=twitter_time(soup1)
    dictio['Text']=twitter_tweets(soup1)
    react=twitter_react(soup1)
    dictio['Replies']=react[0]
    dictio['Reposts']=react[1]
    dictio['Likes']=react[2]
    dictio['Bookmarks']=react[3]
    dictio['Views']=react[4]
    
    data = pd.DataFrame(dictio)
    
    return data

'''Fonction principale :'''

def Twitter_scrapping(nombre_iter,date_depart, date_fin , keyword):
    
    recherche = keyword+' lang:en until:'+date_fin+' since:'+date_depart
    
    url = 'https://twitter.com/explore'
    options = Options()
    options.add_argument("--window-size=1920,1080")
    tweets_df=pd.DataFrame()
    
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    # Wait for the login input field to be present
    wait = WebDriverWait(driver, 20)
    log = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@autocomplete='username']")))
    # Input the email
    log.send_keys('mana_isma@hotmail.com')
    log.send_keys(Keys.RETURN)
    
    # Input the pseudo
    pseudo = wait.until(EC.presence_of_element_located((By.XPATH, "//input")))
    pseudo.send_keys('Izumiu10')
    pseudo.send_keys(Keys.RETURN)

    # Input the password
    password=wait.until(EC.presence_of_element_located((By.XPATH, "//input[@name='password']")))
    password.send_keys('Izumiii0')
    password.send_keys(Keys.RETURN)


    #Search
    search = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@aria-label='Search query']")))
    search.send_keys(str(recherche))

    search.send_keys(Keys.RETURN)
    

    time.sleep(10)
    SCROLL_PAUSE_TIME = 20

    itera = 0

    last_height = driver.execute_script("return document.body.scrollHeight")

    while itera < int(nombre_iter):

        page_content = driver.page_source

        soup = BeautifulSoup(page_content, 'html.parser')

        try:
        # Code susceptible de lever une exception
            df = exctract_twitter_data(soup)
            tweets_df = pd.concat([tweets_df, df], ignore_index=True)
        except TypeError as e:
            # Traitement de l'erreur TypeError
            print("Une erreur s'est produite :", e)
        except KeyError :
            print("Erreur de keyerror")

        itera+=1

        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            itera+=2*int(nombre_iter)/10
        last_height = new_height

    driver.quit()

    return tweets_df


In [3]:
import nltk
#from nltk.tokenize import regexp_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import stopwords # import English stopwords
#from nltk.stem import PorterStemmer
#from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')

stopwords = set(stopwords.words('english'))


import spacy

from textblob import TextBlob


def convert_to_date(date_str):
    try:
        # Essayer de convertir en utilisant le format 1
        return datetime.strptime(date_str, "%b %d, %Y · %I:%M %p %Z")
    except ValueError:
        try:
            # Essayer de convertir en utilisant le format 2
            return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ")
        except ValueError:
            # Gérer le cas où la chaîne n'est dans aucun des formats attendus
            print(f"Format de date non reconnu: {date_str}")
            return None

def columns_date(df):
    df['Date'] = df['Date'].apply(convert_to_date)
    df['Year']=df['Date'].dt.year
    df['Month']=df['Date'].dt.month
    df['Day']=df['Date'].dt.day
    df = df.drop('Date',axis=1)

# Remove all emojis, replace by EMOJI
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Remove @ and mention, replace by USER
def remove_mention(text):
    at=re.compile(r'@\S+')
    return at.sub(r'',text)


# Remove numbers, replace it by NUMBER
def remove_number(text):
    num = re.compile(r'[-+]?[.\d]*[\d]+[:,.\d]*')
    return num.sub(r'', text)


# Remove all URLs, replace by URL
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)


# Factorize repeated punctuation, add REPEAT
def remove_repeat_punct(text):
    rep = re.compile(r'([!?.]){2,}')
    return rep.sub(r'\1', text)


# Factorize elongated words, add ELONG
def remove_elongated_words(text):
    rep = re.compile(r'\b(\S*?)([a-z])\2{2,}\b')
    return rep.sub(r'\1\2', text)


# Remove words in capslock, add ALLCAPS
def remove_allcaps(text):
    caps = re.compile(r'([^a-z0-9()<>\'`\-]){2,}')
    return caps.sub(r'', text)


# Remove all english stopwords
def remove_stopwords(text):
    text = ' '.join([word for word in text.split() if word not in (stopwords)])
    return text


# Remove all punctuations
def remove_all_punct(text):
    table = str.maketrans('','',string.punctuation)
    return text.translate(table)


# Remove punctuations
def remove_punct(text):
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" 
    for p in punctuations:
        text = text.replace(p, f' {p} ')

    text = text.replace('...', ' ... ')
    if '...' not in text:
        text = text.replace('..', ' ... ')   
    return text

# Remove non printable characters
def remove_not_ASCII(text):
    text = ''.join([word for word in text if word in string.printable])
    return text


words_model = ['israeli','israelis','lsrael','endisraelsgenocide','palestinian','palestinians','gazaunderattack']
model_language = {'israel':['israeli','israelis','lsrael','endisraelsgenocide'],
                  'palestine':['palestinian','palestinians'],
                 'gaza':['gazaunderattack']}

def clean_lemma(text):
    data = text.split(' ')
    n=len(data)
    i=0
    while i < n:
        if len(data[i])<=2:
            data.pop(i)
            n-=1
        elif data[i] in words_model:
            for key,value in model_language.items():
                if data[i] in value:
                    data[i]=key
                    i+=1
                    break
        else:
            i+=1
    text = ' '.join(data)
    return text

nlp = spacy.load('en_core_web_md')

def lemmatizing2(text):
    # Process the text using spaCy
    doc = nlp(text)

    # Extract lemmatized tokens
    lemmatized_tokens = [token.lemma_ for token in doc]

    # Join the lemmatized tokens into a sentence
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text

# Clean text
def clean_tweet(text):
    
    # Remove non text
    text = remove_URL(text)      #remove links
    text = remove_not_ASCII(text)      #remove nonsense character
    
    # Lower text, replace abbreviations
    text = text.lower()          #replace all word to lower
    text = remove_mention(text)  #remove @--
    text = remove_number(text)   #remove numbers
    
    # Remove emojis / smileys
    text = remove_emoji(text) 
    
    # Remove repeated puntuations / words
    text = remove_elongated_words(text)    #revomve word repeated like HHHHHH
    text = remove_repeat_punct(text)       #remove repeated punctuation

    text = remove_all_punct(text)          #remove punctuation
    text = remove_punct(text)              #remove other punctuation
    text = remove_stopwords(text)          #remove basic words like (and,or,...)
    
    #Stemming or Lemmatization
    text = lemmatizing2(text)               #replace words to simple (infinitif , singular, ...)
    
    #My methode of lemmatization
    text =clean_lemma(text)

    return text


def polarity(text):
    return TextBlob(text).sentiment.polarity

def sentiment(label):
    if label <0:
        return -1
    elif label ==0:
        return 0
    elif label>0:
        return 1

def sentiment_data(df):
    df['Sentiment_polarity']=df['Text'].apply(polarity)
    df['Sentiment']=df['Sentiment_polarity'].apply(sentiment)

def most_common_words(df):
    texts=''
    for text in df['Text']:
        texts=texts+' '+text
    test = texts.split(' ')
    c=Counter(test)
    return c.most_common(200)

def create_feature_occurrence_table(comments, features):
    # Initialize an empty dictionary to store the feature counts
    feature_counts = {}

    # Iterate over each feature
    for feature in features:
        # Initialize an empty list to store the occurrence counts for the feature
        counts = []

        # Iterate over each comment
        for comment in comments:
            # Count the occurrences of the feature in the comment
            count = comment.count(feature)
            # Append the count to the list
            counts.append(count)

        # Store the list of counts in the feature_counts dictionary
        feature_counts[feature] = counts

    # Create a DataFrame from the feature_counts dictionary
    df = pd.DataFrame(feature_counts)

    # Return the DataFrame
    return df

def text_to_feature(df):

    columns=[]
    for i in most_common_words(df):
        columns.append(i[0])

    texts=list(df['Text'])

    return create_feature_occurrence_table(texts,columns)

def clean_data(df):
    columns_date(df)
    df = df.drop(['Username','Date'],axis=1)
    df['Text']=df['Text'].apply(clean_tweet)
    df_occurrence = text_to_feature(df)
    df = pd.concat([df,df_occurrence] , axis=1)
    sentiment_data(df)
    df_filtered = df.loc[~(df.iloc[:,8:-2] == 0).all(axis=1)]

    return df_filtered

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\perso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import pandas as pd

def Visuel_data(df):
    df2 = df[['Day','Sentiment']]

    # Compter le nombre de chaque sentiment pour chaque année
    sentiment_counts = df2.groupby(['Day', 'Sentiment']).size().unstack(fill_value=0)

    # Assurez-vous que toutes les valeurs de sentiment sont présentes dans le DataFrame
    sentiment_counts = sentiment_counts.reindex(columns=[-1, 0, 1], fill_value=0)

    # Définir les positions des barres
    day = sentiment_counts.index
    n_day = len(day)
    n_bars = len(sentiment_counts.columns)
    bar_width = 0.3
    x = np.arange(n_day)

    # Tracer l'histogramme avec des barres groupées
    fig, ax = plt.subplots(figsize=(12, 8))  # Agrandir la taille du plot à 12x8 pouces
    for i, sentiment in enumerate(sentiment_counts.columns):
        if sentiment < 0 :
            ax.bar(x + i * bar_width, sentiment_counts[sentiment], width=bar_width, color='red' ,label='Sentiment Négatif')
        elif sentiment == 0 :
            ax.bar(x + i * bar_width, sentiment_counts[sentiment], width=bar_width, color='gray' ,label='Sentiment Neutre')
        else :
            ax.bar(x + i * bar_width, sentiment_counts[sentiment], width=bar_width, color='green' ,label='Sentiment Positif')

    # Ajouter des étiquettes et un titre
    ax.set_xlabel('Day')
    ax.set_ylabel('Number of Sentiments')
    ax.set_title('Number of Each Sentiment by Year')
    ax.set_xticks(x + bar_width * (n_bars - 1) / 2)
    ax.set_xticklabels(day)
    ax.legend(title='Sentiment')

    # Afficher le graphique
    plt.savefig('static/images/sentiment_plot.png')



def Multiple_Visuel_data(path_csv):
    df = pd.read_csv(path_csv)

    df2 = df[['Day','Sentiment']]
    
    # Compter le nombre de chaque sentiment pour chaque année
    sentiment_counts = df2.groupby(['Day', 'Sentiment']).size().unstack(fill_value=0)

    # Assurez-vous que toutes les valeurs de sentiment sont présentes dans le DataFrame
    sentiment_counts = sentiment_counts.reindex(columns=[-1, 0, 1], fill_value=0)

    # Définir les positions des barres
    days = sentiment_counts.index
    n_days = len(days)
    n_bars = len(sentiment_counts.columns)
    bar_width = 0.3
    x = np.arange(n_days)

    # Définir des couleurs pour chaque sentiment
    colors = { -1: 'red', 0: 'gray', 1: 'green' }

    # Créer une figure avec plusieurs sous-graphiques
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))  # Créer une grille 2x2 de sous-graphiques

    # Premier plot
    axs[0, 0].bar(x - bar_width, sentiment_counts[-1], width=bar_width*2, color=colors[-1], label='Sentiment -1')
    axs[0, 0].set_title('Sentiment Négatifs')
    axs[0, 0].set_xticks(x)
    axs[0, 0].set_xticklabels(days)
    axs[0,0].set_xlabel('Day')
    axs[0,0].set_ylabel('Number of Sentiments')

    # Deuxième plot
    axs[0, 1].bar(x, sentiment_counts[0], width=bar_width*2, color=colors[0], label='Sentiment 0')
    axs[0, 1].set_title('Sentiment Neutre')
    axs[0, 1].set_xticks(x)
    axs[0, 1].set_xticklabels(days)
    axs[0,1].set_xlabel('Day')
    axs[0,1].set_ylabel('Number of Sentiments')

    # Troisième plot
    axs[1, 0].bar(x + bar_width, sentiment_counts[1], width=bar_width*2, color=colors[1], label='Sentiment 1')
    axs[1, 0].set_title('Sentiment Positif')
    axs[1, 0].set_xticks(x)
    axs[1, 0].set_xticklabels(days)
    axs[1,0].set_xlabel('Day')
    axs[1,0].set_ylabel('Number of Sentiments')

    # Quatrième plot: Histogramme empilé
    for i, sentiment in enumerate(sentiment_counts.columns):
        axs[1, 1].bar(x + i * bar_width, sentiment_counts[sentiment], width=bar_width, color=colors[sentiment], label=f'Sentiment {sentiment}')
    axs[1, 1].set_title('Histogramme Empilé')
    axs[1, 1].set_xticks(x + bar_width)
    axs[1, 1].set_xticklabels(days)
    axs[1, 1].legend(title='Sentiment')
    axs[1,1].set_xlabel('Day')
    axs[1,1].set_ylabel('Number of Sentiments')

    # Ajuster l'espacement entre les sous-graphiques
    plt.tight_layout()

    # Afficher le graphique
    plt.savefig('static/images/sentiment_plot3.png')

In [20]:
df = pd.DataFrame()

In [21]:
for year in ['2024']:
    for month in range(2,7):
        for keyword in ['palestine','israel','gaza','genocide']:
            date_fin = year+'-'+str(month)+'-'+'01'
            print(date_fin+'-'*20)
            date_depart = '2023-12-31'
            df_scraped = Twitter_scrapping(20,date_depart, date_fin , keyword)
            df = pd.concat([df,df_scraped],ignore_index = True)


2024-2-01--------------------
/mostdastardly
/GlobeEyeNews
/k1llacourt
/CensoredMen
/IvyTaq
Une erreur s'est produite : 'NoneType' object is not subscriptable
/sgb7262
Une erreur s'est produite : 'NoneType' object is not subscriptable
/REVMAXXING
/PalestineCultu1
/ContraryNotions
/REVMAXXING
/EndWokeness
/zaramagnusson
/REVMAXXING
/REVMAXXING
/yourexgirltara
/StopAntisemites
/REVMAXXING
/DrLoupis
/Prolotario1
/mihaschw
/REVMAXXING
/visegrad24
/Palestine001_
/DrLoupis
/extracapsa
/ShaykhSulaiman
/InfraHaz
/SuppressedNws
/jacksonhinklle
/brfootball
/StopAntisemites
/EndWokeness
/PALMENA_IC
/MyLordBebo
/DrLoupis
/HumaZhr
/StopAntisemites
/EndWokeness
/CensoredMen
/_positivity___
/tazen1000
Une erreur s'est produite : 'NoneType' object is not subscriptable
/HenMazzig
/mm_panorama
Une erreur s'est produite : 'NoneType' object is not subscriptable
/LozzaFox
/jacksonhinklle
/G0BACKAROUND
Une erreur s'est produite : 'NoneType' object is not subscriptable
/jacksonhinklle
/SaraReyi
/MyLordBebo
/

/visegrad24
/BlakPantherBabe
/Resist_05
/janet_rice
/CensoredMen
/CensoredMen
/MrSinha_
/GMF_Palestine
/PopBase
/QudsNen
/2RawTooReal
/QudsNen
/2RawTooReal
/ayca_cu
/DrEliDavid
/POTUS
/PBDsPodcast
2024-3-01--------------------
/CensoredMen
/MOSSADil
/stairwayto3dom
/Toronto4P
/expatvibes
/KrisSkulls
/broseph_stalin
/stairwayto3dom
/Cancelcloco
/GlobeEyeNews
/DocumentIsrael
/indianaftali
/AzzatAlsaalem
/stairwayto3dom
/Resist_05
/GlobeEyeNews
/persianjewess
/S_O_F_I_E__
/HananyaNaftali
/sahouraxo
/jacksonhinklle
/paulrubens
/LozzaFox
/IDF
/ClareDalyMEP
/OliLondonTV
/InfraHaz
/IDF
/SuppressedNws
/desireerugani
/AuschwitzMuseum
/CensoredMen
/jacksonhinklle
/BRICSinfo
/jakeshieldsajj
/jakeshieldsajj
/IDF
/AssalRad
/DrEliDavid
/ShaykhSulaiman
/StopAntisemites
/IDF
/Salwan_Momika1
/VictorIsrael_
/NizMhani
/DrLoupis
/EylonALevy
/CensoredMen
/Prolotario1
/RealAlexJones
Une erreur s'est produite : 'NoneType' object is not subscriptable
/Muqadaam
/ukraine_map
/MarinaMedvin
/IDF
/TorahJudaism
/an

/etanadeeIL
/visegrad24
/BNE094
Une erreur s'est produite : 'NoneType' object is not subscriptable
/BTnewsroom
/elonmusk
/AssalRad
/SenSanders
/israeljauregui
Une erreur s'est produite : 'NoneType' object is not subscriptable
/DrEliDavid
/martyrmade
Une erreur s'est produite : 'NoneType' object is not subscriptable
/BeckettUnite
/BeckettUnite
/OwenJones84
/jakeshieldsajj
Une erreur s'est produite : 'NoneType' object is not subscriptable
2024-4-01--------------------
/InsiderWorld_1
/sahouraxo
/mhdksafa
/MyLordBebo
/SuppressedNws
/Kahlissee
/SuppressedNws
/ShaykhSulaiman
/brianoflondon
/narrative_hole
/SuppressedNws
/OsintExperts
/NewhamIndParty
/alconanonymous
/DailyLoud
/ShaykhSulaiman
/persianjewess
/Kahlissee
/YuvalMimi
/Kahlissee
/darrengrimes_
/Hind_Gaza
/OliLondonTV
/marionumber4
Une erreur s'est produite : 'NoneType' object is not subscriptable
/ShaykhSulaiman
/QudsNen
/YosephHaddad
/jakeshieldsajj
/Israel
/ytirawi
/darrengrimes_
/BRICSinfo
/mhdksafa
/noatishby
/HananyaNaftali
/

/jacksonhinklle
/MrAndyNgo
/BernieSanders
/QudsNen
/SenSanders
/SenSanders
/RepBrianMast
/amnesty
/RepRashida
/medeabenjamin
/MrAndyNgo
/Kahlissee
/Kahlissee
/QudsNen
/21WIRE
/sciaticus_finch
/SenSanders
/hotpinkIwt
Une erreur s'est produite : 'NoneType' object is not subscriptable
/NationwideRadio
/BTnewsroom
/CherylWroteIt
/aziz0nomics
/tparsi
/OwenJones84
/MiddleEastEye
/BTnewsroom
/AP
/Hind_Gaza
/Pontifex
/ionebelarra
/mehdirhasan
2024-5-01--------------------
/stairwayto3dom
/SuppressedNws
/sahouraxo
/AzzatAlsaalem
/dyansbuffa
/miavinesbooth
/Resist_05
/gregjstoker
/sahouraxo
/damekatydenise_
/mhdksafa
/ShaykhSulaiman
/PatriciaChimie1
/OliLondonTV
/mhdksafa
/MuhammadSmiry
/bruceaswanton
/HotSpotHotSpot
/ismail_i_ak
/BellamSwathi
/jacksonhinklle
/CensoredMen
/SunBirdZAKH
Une erreur s'est produite : 'NoneType' object is not subscriptable
/mhdksafa
/sahouraxo
/Workers4Pal
/Megatron_ron
/Kahlissee
/KimDotcom
/sahouraxo
/YusufCatStevens
/Kahlissee
/21WIRE
/Joe__Bassey
/kirawontmiss
Une

/HotSpotHotSpot
/safetyzonn
Une erreur s'est produite : 'NoneType' object is not subscriptable
/postnuclearjoan
Une erreur s'est produite : 'NoneType' object is not subscriptable
/EitanChitayat
/KerryBurgess
/Starboy2079
/HadiNasrallah
Une erreur s'est produite : 'NoneType' object is not subscriptable
/Resist_05
/Partisangirl
/KonstantinTeam
Une erreur s'est produite : 'NoneType' object is not subscriptable
/DrLoupis
/giftedswifted
/StopAntisemites
/sahouraxo
/Kahlissee
/fizaiq
/Partisangirl
/DillyHussain88
Une erreur s'est produite : 'NoneType' object is not subscriptable
/SanaSaeed
/hinduoncampus
/liz_churchill10
/ShaykhSulaiman
Une erreur s'est produite : 'NoneType' object is not subscriptable
/ShaykhSulaiman
/CrimeWatchMpls
/NokhaDakroub
/leiaversacespam
/TheIndMuslim
/ChinasaNworu
/peterdaou
/ShaykhSulaiman
/itgirlgaga
/NiohBerg
/ShaykhSulaiman
/ManPalestine
/palyouthmvmt
/shipaxee
Une erreur s'est produite : 'NoneType' object is not subscriptable
/medeabenjamin
/taesoothe
/sahour

In [22]:
df

,Username,Date,Text,Replies,Reposts,Likes,Bookmarks,Views
0,mostdastardly,2024-01-31T23:52:22.000Z,( pinned. info below. ),0,4,16,0,845
1,GlobeEyeNews,2024-01-14T17:36:05.000Z,Russian President Vladimir Putin:\n\nPalestine...,485,4809,18032,309,392142
2,k1llacourt,2024-01-11T02:24:23.000Z,Ok i need y’all to watch this whole video & th...,445,11747,26336,10457,3880577
3,REVMAXXING,2024-01-21T04:34:08.000Z,Palestine WILL win!,616,2710,13177,36,152110
4,PalestineCultu1,2024-01-10T22:13:22.000Z,The whole world is proud of you,623,10920,35838,106,570684
...,...,...,...,...,...,...,...,...
1617,peterdaou,2024-05-31T09:53:04.000Z,DOES BIDEN WANT WWIII?\n\nJust in the past 48 ...,941,2013,4373,138,207017
1618,ShaykhSulaiman,2024-05-29T14:59:00.000Z,RASHIA TLIAB CONDEMNS JOE BIDEN FOR COMPLICITY...,36,113,266,4,11376
1619,itgirlgaga,2024-05-29T14:59:51.000Z,Hello little monsters! As we all know gaga has...,119,325,1181,45,91806
1620,NiohBerg,2024-05-30T13:46:11.000Z,IDF makes sure nobody gets back into Jabaliya ...,12,47,359,12,11012


In [23]:
df2 = pd.read_csv('Data_tweets.csv')
df2 = df2.drop(['Unnamed: 0'],axis=1)
df2

,Username,Date,Text,Replies,Reposts,Likes,Bookmarks,Views
0,localdiscoclub,2023-12-31T23:59:25.000Z,this is how i chose to spend my new year’s eve...,6,15,43,0,2042
1,redsarah99,2023-12-31T23:59:27.000Z,Here for Palestine,0,13,42,0,843
2,PalestineCultu1,2023-12-11T06:00:01.000Z,Who stands with me?,919,5755,14211,58,213650
3,freemonotheist,2023-12-16T19:55:10.000Z,The Beauty of Palestine Bethlehem in 1880,781,8236,26766,417,816266
4,StopZionistHate,2023-12-28T02:18:10.000Z,"""...In Palestine, there is no one innocent, ok...",489,2505,5978,343,188269
...,...,...,...,...,...,...,...,...
3103,RepMoBrooks,2019-08-15T18:56:26.000Z,"#Israel \n\nLike many nations, Israel bars ene...",7839,3796,10025,9,0
3104,newtgingrich,2020-08-13T17:43:24.000Z,While Biden was hiding in the basement Preside...,964,5777,23053,13,0
3105,VictorIsrael_,2020-09-20T19:04:35.000Z,"Lmao Ozo\n\nDem Evict you, you dey find kiss, ...",246,756,3838,4,0
3106,SecPompeo,2020-11-19T12:48:42.000Z,Proud to support American policy and stand wit...,421,808,4646,12,0


In [24]:
df_tweets = df.drop_duplicates()
df_tweets = pd.concat([df2,df_tweets],ignore_index = True)
df_tweets

,Username,Date,Text,Replies,Reposts,Likes,Bookmarks,Views
0,localdiscoclub,2023-12-31T23:59:25.000Z,this is how i chose to spend my new year’s eve...,6,15,43,0,2042
1,redsarah99,2023-12-31T23:59:27.000Z,Here for Palestine,0,13,42,0,843
2,PalestineCultu1,2023-12-11T06:00:01.000Z,Who stands with me?,919,5755,14211,58,213650
3,freemonotheist,2023-12-16T19:55:10.000Z,The Beauty of Palestine Bethlehem in 1880,781,8236,26766,417,816266
4,StopZionistHate,2023-12-28T02:18:10.000Z,"""...In Palestine, there is no one innocent, ok...",489,2505,5978,343,188269
...,...,...,...,...,...,...,...,...
4694,peterdaou,2024-05-31T09:53:04.000Z,DOES BIDEN WANT WWIII?\n\nJust in the past 48 ...,941,2013,4373,138,207017
4695,ShaykhSulaiman,2024-05-29T14:59:00.000Z,RASHIA TLIAB CONDEMNS JOE BIDEN FOR COMPLICITY...,36,113,266,4,11376
4696,itgirlgaga,2024-05-29T14:59:51.000Z,Hello little monsters! As we all know gaga has...,119,325,1181,45,91806
4697,NiohBerg,2024-05-30T13:46:11.000Z,IDF makes sure nobody gets back into Jabaliya ...,12,47,359,12,11012


In [25]:
df_tweets.to_csv('Data_tweets.csv')

In [29]:
df_cleaned = df_tweets.copy()

In [31]:
df_cleaned = clean_data(df_tweets)

In [32]:
df_cleaned

,Text,Replies,Reposts,Likes,Bookmarks,Views,Year,Month,Day,israel,...,defend,freepalestine,middle,antisemitic,ever,military,action,law,Sentiment_polarity,Sentiment
0,choose spend new year eve disappointed governm...,6,15,43,0,2042,2023,12,31,0,...,0,0,0,0,0,0,0,0,-0.159091,-1
1,palestine,0,13,42,0,843,2023,12,31,0,...,0,0,0,0,0,0,0,0,0.000000,0
2,stand,919,5755,14211,58,213650,2023,12,11,0,...,0,0,0,0,0,0,0,0,0.000000,0
3,beauty palestine bethlehem,781,8236,26766,417,816266,2023,12,16,0,...,0,0,0,0,0,0,0,0,0.000000,0
4,palestine one innocent okay need radically ext...,489,2505,5978,343,188269,2023,12,28,0,...,0,0,0,0,0,0,0,0,0.500000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4694,biden want wwi past hour support genocide gaza...,941,2013,4373,138,207017,2024,5,31,0,...,0,0,0,0,0,0,0,0,-0.250000,-1
4695,rashia tliab condemn joe biden complicity geno...,36,113,266,4,11376,2024,5,29,0,...,0,0,0,0,0,0,0,0,0.000000,0
4696,hello little monster know gaga silent palestin...,119,325,1181,45,91806,2024,5,29,0,...,0,0,0,0,0,0,0,0,-0.195000,-1
4697,idf make sure nobody get back jabaliya every s...,12,47,359,12,11012,2024,5,30,0,...,0,0,0,0,2,0,0,0,0.142857,1


In [34]:
most_common_words(df_cleaned)

[('israel', 2941),
 ('palestine', 2050),
 ('gaza', 1000),
 ('not', 528),
 ('genocide', 496),
 ('people', 460),
 ('say', 376),
 ('support', 316),
 ('today', 295),
 ('one', 286),
 ('hamas', 267),
 ('break', 265),
 ('state', 261),
 ('world', 261),
 ('east', 257),
 ('child', 251),
 ('free', 245),
 ('year', 244),
 ('call', 243),
 ('day', 240),
 ('get', 235),
 ('war', 228),
 ('stand', 224),
 ('make', 224),
 ('like', 217),
 ('kill', 215),
 ('see', 207),
 ('right', 205),
 ('country', 196),
 ('take', 195),
 ('president', 194),
 ('trump', 190),
 ('new', 183),
 ('time', 181),
 ('peace', 177),
 ('terrorist', 169),
 ('stop', 167),
 ('attack', 161),
 ('want', 156),
 ('jewish', 156),
 ('show', 156),
 ('know', 146),
 ('biden', 140),
 ('protest', 139),
 ('never', 134),
 ('would', 134),
 ('use', 134),
 ('jews', 132),
 ('come', 129),
 ('first', 129),
 ('bomb', 128),
 ('give', 124),
 ('love', 124),
 ('tell', 122),
 ('woman', 121),
 ('family', 121),
 ('end', 117),
 ('aid', 117),
 ('friend', 116),
 ('last',

In [35]:
df_cleaned.drop(['Text'],axis=1).to_csv('Data_tweets_cleaned.csv')